##### Import


In [1]:
# when in google drive
from google.colab import drive
drive.mount('/content/gdrive')
!pip install scikeras --quiet

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from keras.utils.np_utils import to_categorical 
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization,\
Flatten, LSTM
# from scikeras.wrappers import KerasClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

NUM_FEATURES = 31
TIMESTEPS = 1
DATASET_LINK = "https://raw.githubusercontent.com/ehandywhyy/ict3204-security-analytics/main/dataset/overall.csv"
TEST_DATA_LINK = "https://raw.githubusercontent.com/ehandywhyy/ict3204-security-analytics/main/dataset/overall_test.csv"

##### Initialise Seed

In [3]:
# random seed for reproducibility
seed = 10
np.random.seed(seed)

# loading of dataset
df = pd.read_csv(DATASET_LINK)

# # Remove missing values IF AVAILABLE and print head
df = df.dropna()
df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,Andy,1,1,0.142176,0.156880,0.014704,0.127937,0.142299,0.014361,0.155919,0.116134,-0.039784,0.124244,0.276458,0.152215,0.137177,0.641011,0.503834,0.125197,0.124527,-0.000670,0.122092,0.100097,-0.021995,0.128263,0.085967,-0.042296,0.120442,0.209372,0.088930,0.116911,0.165437,0.048526,0.160258
1,Andy,1,2,0.141793,0.164635,0.022842,0.121625,0.152146,0.030520,0.137039,0.121543,-0.015496,0.104216,0.218603,0.114388,0.156909,0.346147,0.189238,0.138544,0.156244,0.017699,0.120454,0.088308,-0.032146,0.144490,0.082397,-0.062093,0.101188,0.126066,0.024878,0.098424,0.130661,0.032238,0.173819
2,Andy,1,3,0.157152,0.147220,-0.009932,0.134796,0.145790,0.010994,0.124892,0.101461,-0.023431,0.112165,0.202102,0.089937,0.140860,0.274559,0.133699,0.142558,0.309424,0.166866,0.121442,0.101698,-0.019745,0.135452,0.082423,-0.053029,0.108598,0.155843,0.047244,0.118759,0.146550,0.027791,0.173512
3,Andy,1,4,0.158673,0.142487,-0.016186,0.118908,0.159179,0.040271,0.133175,0.060895,-0.072279,0.106036,0.333517,0.227481,0.145889,0.287380,0.141491,0.125951,0.136230,0.010279,0.108067,0.114496,0.006429,0.114266,0.072556,-0.041711,0.101937,0.145692,0.043755,0.096300,0.122580,0.026280,0.190808
4,Andy,1,5,0.166583,0.157102,-0.009480,0.115094,0.141103,0.026009,0.135121,0.082097,-0.053024,0.128655,0.192888,0.064234,0.165007,0.286537,0.121530,0.126328,0.119997,-0.006331,0.113122,0.082637,-0.030484,0.125474,0.089247,-0.036227,0.114211,0.149136,0.034925,0.121197,0.230113,0.108917,0.143299


##### Divide dataset into X and Y
##### Normalise features within range 0 (minimum) and 1 (maximum)


In [4]:
dataset = df.values

# divide data into features X and target (Classes) Y
X = dataset[:,3:].astype(float)
Y = dataset[:,0]

# # check for class imbalance
print(df.groupby(Y).size())

Andy       100
Azfar      100
Chris      100
Qikai      100
Safaraz    100
dtype: int64


In [5]:
# convert target Y to one hot encoded Y for model
Y = Y.reshape(-1, 1)
encoder = OneHotEncoder().fit(Y)

# get all the encoded class
print(encoder.get_feature_names_out())

# print X and Y shape
print("X dataset shape: " + str(X.shape))
print("Y dataset shape: " + str(Y.shape))

['x0_Andy' 'x0_Azfar' 'x0_Chris' 'x0_Qikai' 'x0_Safaraz']
X dataset shape: (500, 31)
Y dataset shape: (500, 1)


##### Preparing dataset

In [6]:
# split dataset into train and test of 0.8/0.2 ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=seed)

# normalisation to 0 to 1
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# reshaping the dataset to include LSTM
X_train = np.asarray(X_train, dtype=np.float32)
X_train = np.reshape(X_train, (X_train.shape[0], TIMESTEPS, X_train.shape[1]))
X_test = np.asarray(X_test, dtype=np.float32)
X_test = np.reshape(X_test, (X_test.shape[0], TIMESTEPS, X_test.shape[1]))

# converting y data to encoding
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

num_classes = y_train.shape[1]

print("X train shape: " + str(X_train.shape))
print("Y train shape: " + str(y_train.shape))
print("X test shape: " + str(X_test.shape))
print("Y test shape: " + str(y_test.shape))

X train shape: (400, 1, 31)
Y train shape: (400, 5)
X test shape: (100, 1, 31)
Y test shape: (100, 5)


### Create Model

In [7]:
def create_model():
    # define model
    model = Sequential()
    model.add(LSTM(units=128, return_sequences=True, 
                 input_shape=(TIMESTEPS,NUM_FEATURES)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(units=64, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    # Softmax for multi-class classification
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam',
                metrics=['accuracy'])
    return model

##### Wrap Model in KerasClassifier

In [8]:
model = KerasClassifier(build_fn=create_model, epochs=100, 
                            batch_size=10)

### Perform KFold Validation

In [9]:
num_folds = 10
kfold = KFold(n_splits=num_folds, 
              shuffle=True,
              random_state=seed)

##### Get Accuracy from KFold Validation

In [10]:
results = cross_val_score(model, X_train, y_train, 
                          cv=kfold, error_score="raise", verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/100
36/36 [==============================] - 6s 10ms/step - loss: 1.0015 - accuracy: 0.6139
Epoch 2/100
36/36 [==============================] - 0s 11ms/step - loss: 0.3985 - accuracy: 0.8389
Epoch 3/100
36/36 [==============================] - 0s 10ms/step - loss: 0.3657 - accuracy: 0.8667
Epoch 4/100
36/36 [==============================] - 0s 10ms/step - loss: 0.2785 - accuracy: 0.8917
Epoch 5/100
36/36 [==============================] - 0s 9ms/step - loss: 0.3091 - accuracy: 0.8889
Epoch 6/100
36/36 [==============================] - 0s 10ms/step - loss: 0.2587 - accuracy: 0.8944
Epoch 7/100
36/36 [==============================] - 0s 10ms/step - loss: 0.3343 - accuracy: 0.8750
Epoch 8/100
36/36 [==============================] - 0s 9ms/step - loss: 0.2800 - accuracy: 0.9111
Epoch 9/100
36/36 [==============================] - 0s 9ms/step - loss: 0.3925 - accuracy: 0.8583
Epoch 10/100
36/36 [==============================] - 0s 9ms/step - loss: 0.2499 - accuracy: 0.9250
Epo

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  8.6min finished


##### get validation accuracy

In [11]:
print("Validation Accuracy of %.2f%% (with standard deviation of %.2f%%)" % 
      (results.mean()*100, results.std()*100))

Validation Accuracy of 98.00% (with standard deviation of 1.87%)


##### fit model

In [12]:
# fit the model
es = EarlyStopping(monitor='loss', mode='min', min_delta=0.001, patience=50,
                   verbose=0)
model.fit(X_train, y_train, callbacks=es)

Epoch 1/100
40/40 [==============================] - 6s 9ms/step - loss: 1.0066 - accuracy: 0.6200
Epoch 2/100
40/40 [==============================] - 0s 9ms/step - loss: 0.4695 - accuracy: 0.8225
Epoch 3/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3953 - accuracy: 0.8400
Epoch 4/100
40/40 [==============================] - 0s 9ms/step - loss: 0.4279 - accuracy: 0.8550
Epoch 5/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2890 - accuracy: 0.8950
Epoch 6/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3048 - accuracy: 0.8875
Epoch 7/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3295 - accuracy: 0.8850
Epoch 8/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2987 - accuracy: 0.8950
Epoch 9/100
40/40 [==============================] - 0s 10ms/step - loss: 0.2642 - accuracy: 0.8975
Epoch 10/100
40/40 [==============================] - 0s 9ms/step - loss: 0.2623 - accuracy: 0.9125
Epoch 11

In [13]:
# view model summary
model.model.summary()

# save model
model.model.save(
    F"/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
# model.model.save("model/key_classifier.h5")

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 1, 128)            81920     
                                                                 
 dropout_30 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_30 (Bat  (None, 1, 128)           512       
 chNormalization)                                                
                                                                 
 lstm_31 (LSTM)              (None, 1, 128)            131584    
                                                                 
 dropout_31 (Dropout)        (None, 1, 128)            0         
                                                                 
 batch_normalization_31 (Bat  (None, 1, 128)           512       
 chNormalization)                                    

In [14]:
# load model for Google Drive
model.model = load_model("/content/gdrive/My Drive/Colab Notebooks/overall_key_classifier.h5")
# model.model = load_model("model/key_classifier.h5")

y_pred = model.predict(X_test)
y_pred = to_categorical(y_pred)

# evaluate predictions
acc = accuracy_score(y_test, y_pred)
print("Testing accuracy: %.3f%%" % (acc*100)

Testing accuracy: 0.960


In [15]:
# import unseen data to check if model works
pred_df = pd.read_csv(TEST_DATA_LINK)
pred_df.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,DD.i.e,UD.i.e,H.e,DD.e.five,UD.e.five,H.five,DD.five.Shift.r,UD.five.Shift.r,H.Shift.r,DD.Shift.r.o,UD.Shift.r.o,H.o,DD.o.a,UD.o.a,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,Andy,1,1,0.174239,0.409866,0.235627,0.133840,0.171659,0.037820,0.156984,0.120599,-0.036385,0.111543,0.390015,0.278472,0.134937,0.264941,0.130004,0.144750,0.115824,-0.028925,0.120440,0.104092,-0.016348,0.137544,0.091721,-0.045824,0.099726,0.099142,-0.000585,0.120912,0.135934,0.015022,0.163508
1,Andy,1,2,0.145289,0.154851,0.009562,0.131217,0.129134,-0.002083,0.130980,0.092216,-0.038764,0.118131,0.191068,0.072937,0.164665,0.306797,0.142132,0.145192,0.127395,-0.017797,0.132026,0.117957,-0.014069,0.164865,0.111584,-0.053281,0.101935,0.023678,-0.078258,0.140863,0.192325,0.051462,0.156958
2,Andy,1,3,0.122266,0.136321,0.014055,0.114905,0.125308,0.010402,0.144332,0.098220,-0.046112,0.112351,0.198390,0.086040,0.121758,0.229243,0.107485,0.142623,0.143437,0.000814,0.120015,0.097180,-0.022835,0.145501,0.095982,-0.049520,0.121049,0.091486,-0.029563,0.111422,0.145751,0.034329,0.169304
3,Andy,1,4,0.121075,0.132562,0.011487,0.134813,0.132049,-0.002763,0.135681,0.079597,-0.056084,0.102607,0.203487,0.100880,0.117718,0.227770,0.110051,0.170992,0.183585,0.012593,0.120631,0.127893,0.007262,0.135072,0.089715,-0.045356,0.117885,0.089209,-0.028676,0.098095,0.139069,0.040974,0.168298
4,Andy,1,5,0.141361,0.124721,-0.016640,0.098268,0.149003,0.050735,0.128713,0.102436,-0.026277,0.101371,0.220777,0.119406,0.112454,0.242917,0.130463,0.144625,0.142355,-0.002269,0.120052,0.152262,0.032211,0.121480,0.078897,-0.042583,0.108607,0.049435,-0.059172,0.131348,0.156398,0.025050,0.170112


In [16]:
pred_dataset = pred_df.values

# divide data into features X
# X_new = new_dataset[:,3:].astype(float)

pred_row=pred_df.iloc[:,3:]
# print("check name")
print(pred_df.iloc[0:7,0:1])

# convert to
pred_row = pred_row.values.tolist()
pred_row = scaler.transform(pred_row)
pred_arr = np.asarray(pred_row, dtype=np.float32)
pred_arr = np.reshape(pred_arr, (pred_row.shape[0], TIMESTEPS, pred_arr.shape[1]))

  subject
0    Andy
1    Andy
2    Andy
3    Andy
4    Andy
5   Azfar
6   Azfar


In [17]:
# get prediction and its label
pred = model.predict(pred_arr)
pred = to_categorical(pred)
pred = encoder.inverse_transform(pred)

pred = np.squeeze(pred)

pred_proba = model.predict_proba(pred_arr)
acc = np.max(pred_proba, axis=1)

print(pred)
print(acc)

['Andy' 'Andy' 'Andy' 'Andy' 'Andy' 'Azfar' 'Azfar' 'Azfar' 'Azfar'
 'Azfar' 'Chris' 'Chris' 'Chris' 'Chris' 'Chris' 'Qikai' 'Qikai' 'Qikai'
 'Qikai' 'Qikai' 'Safaraz' 'Safaraz' 'Safaraz' 'Safaraz' 'Safaraz']
[0.9970372  0.99690634 0.99678135 0.99585354 0.9443761  0.9997373
 0.99990714 0.9997161  0.9998529  0.99986684 0.9141486  0.999801
 0.9993192  0.9994541  0.9957432  0.9997023  0.99755496 0.99392444
 0.9996562  0.99647576 0.99814725 0.99881077 0.99283665 0.97376597
 0.9313083 ]
